In [1]:
import numpy as np
import tensorflow as tf


/Users/jmangia/anaconda2/envs/SwiftNLC/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Import word2vec vectorized traning dataset prepared with FastText

In [2]:
# import intents file
import json
with open('vectorizedDataset.json') as json_data:
    intents = json.load(json_data)

In [83]:
classes = len(intents['intents'])
currentClass = 0
train_x = []
train_y = []

for intent in intents['intents']:
    y = [0] * classes
    y[currentClass] = 1

    for vector in intent['sentenceVectors']:
        train_x.append(vector)
        train_y.append(y)

    currentClass += 1

vectorSize = len(train_x[0])

train_X = np.array(train_x)
train_Y = np.array(train_y)

print(classes, "classes")
print(vectorSize, "vector size")
print(len(train_x), len(train_x[0]), "x")
print(len(train_y), len(train_y[0]), "y")
print(train_X.shape)
print(train_Y.shape)

(8, 'classes')
(100, 'vector size')
(26, 100, 'x')
(26, 8, 'y')
(26, 100)
(26, 8)


# Prepare Keras / TensorFlow model

In [112]:
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Bidirectional, LSTM, GRU, TimeDistributed, Activation, Flatten, Embedding
from keras.optimizers import Adam


In [113]:
#sequence_input = Input(shape=(None, len(train_x[0])), dtype='float')
#bidiGru = Bidirectional(GRU(100))(sequence_input)
#preds = Dense(len(train_y[0]), activation='softmax')(bidiGru)
#model = Model(sequence_input, preds)
#model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])

model = Sequential([
                    GRU(100, batch_size=1, input_shape=(None, len(train_x[0])), return_sequences=True),
                    TimeDistributed(Dense(64)),
                    Activation('relu'),
                    TimeDistributed(Dense(32)),
                    Activation('relu'),
                    TimeDistributed(Dense(len(train_y[0]))),
                    Activation('softmax'),
                   ])

print("model fitting - Bidirectional GRU")
model.summary()


model fitting - Bidirectional GRU
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_44 (GRU)                 (1, None, 100)            60300     
_________________________________________________________________
time_distributed_102 (TimeDi (1, None, 64)             6464      
_________________________________________________________________
activation_97 (Activation)   (1, None, 64)             0         
_________________________________________________________________
time_distributed_103 (TimeDi (1, None, 32)             2080      
_________________________________________________________________
activation_98 (Activation)   (1, None, 32)             0         
_________________________________________________________________
time_distributed_104 (TimeDi (1, None, 8)              264       
_________________________________________________________________
activation_99 (Activation)   (1, None, 8) 

In [114]:
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train Model

In [118]:
print(train_X.shape)
print(train_Y.shape)

X3 = np.reshape(train_X, (1,) + train_X.shape)
Y3 = np.reshape(train_Y, (1,) + train_Y.shape)

print(X3.shape)
print(Y3.shape)

print(X3)


(26, 100)
(26, 8)
(1, 26, 100)
(1, 26, 8)
[[[ 0.03152543 -0.07608503  0.03927452 ... -0.08470749  0.06540044
   -0.09444779]
  [ 0.02298637 -0.00379145 -0.03864405 ...  0.05101557 -0.10139443
   -0.02094366]
  [-0.01482135 -0.08236423 -0.00952277 ...  0.04878472  0.05576552
   -0.04330803]
  ...
  [ 0.04357786 -0.09341566  0.05002676 ... -0.0229912   0.02283798
   -0.01719651]
  [ 0.01132766 -0.05173061  0.03774963 ... -0.00333958 -0.02325496
    0.01082035]
  [ 0.03639279 -0.07542431  0.03309561 ... -0.03817781 -0.01912619
    0.10327344]]]


In [116]:
model.fit(X3, Y3, epochs=100)

Epoch 1/100
1/1 [==============================] - 1s 1s/step - loss: 2.0763 - acc: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 17ms/step - loss: 2.0670 - acc: 0.1923
Epoch 3/100
1/1 [==============================] - 0s 17ms/step - loss: 2.0568 - acc: 0.2692
Epoch 4/100
1/1 [==============================] - 0s 17ms/step - loss: 2.0452 - acc: 0.4615
Epoch 5/100
1/1 [==============================] - 0s 17ms/step - loss: 2.0337 - acc: 0.5385
Epoch 6/100
1/1 [==============================] - 0s 16ms/step - loss: 2.0222 - acc: 0.5385
Epoch 7/100
1/1 [==============================] - 0s 18ms/step - loss: 2.0101 - acc: 0.6154
Epoch 8/100
1/1 [==============================] - 0s 17ms/step - loss: 1.9973 - acc: 0.7308
Epoch 9/100
1/1 [==============================] - 0s 17ms/step - loss: 1.9834 - acc: 0.7308
Epoch 10/100
1/1 [==============================] - 0s 18ms/step - loss: 1.9685 - acc: 0.7308
Epoch 11/100
1/1 [==============================] - 0s 18ms/step - 

Epoch 89/100
1/1 [==============================] - 0s 20ms/step - loss: 0.0080 - acc: 1.0000
Epoch 90/100
1/1 [==============================] - 0s 19ms/step - loss: 0.0078 - acc: 1.0000
Epoch 91/100
1/1 [==============================] - 0s 19ms/step - loss: 0.0075 - acc: 1.0000
Epoch 92/100
1/1 [==============================] - 0s 21ms/step - loss: 0.0073 - acc: 1.0000
Epoch 93/100
1/1 [==============================] - 0s 19ms/step - loss: 0.0071 - acc: 1.0000
Epoch 94/100
1/1 [==============================] - 0s 18ms/step - loss: 0.0069 - acc: 1.0000
Epoch 95/100
1/1 [==============================] - 0s 20ms/step - loss: 0.0067 - acc: 1.0000
Epoch 96/100
1/1 [==============================] - 0s 20ms/step - loss: 0.0065 - acc: 1.0000
Epoch 97/100
1/1 [==============================] - 0s 19ms/step - loss: 0.0063 - acc: 1.0000
Epoch 98/100
1/1 [==============================] - 0s 18ms/step - loss: 0.0061 - acc: 1.0000
Epoch 99/100
1/1 [==============================] - 0s 17ms/

# Export Model using CoreML Tools

In [ ]:
model.save('SwiftNLCFastText.h5')

In [ ]:
import coremltools
coreml_model = coremltools.converters.keras.convert(model, input_names="vectors", output_names="entities")
coreml_model

In [ ]:
coreml_model.save('SwiftNLCFastText.mlmodel')